# Análise de Maturidade de Produtos IoT - TechNova\n\n**Especialista em Data Analytics**  \n**Data:** 2026-01-16\n\n## Objetivo\nIdentificar a melhor startup IoT para investimento baseada em análise multidimensional de maturidade, com foco especial no equilíbrio entre Performance Técnica e Viabilidade Econômica.

In [ ]:
# Importar bibliotecas\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport plotly.graph_objects as go\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# Configurações\nplt.style.use('seaborn-v0_8-darkgrid')\nsns.set_palette('husl')\npd.set_option('display.max_columns', None)

## 1. Tratamento de Dados

In [ ]:
# Carregar dados\ndf = pd.read_excel('Case_TechNova_Dados.xlsx', sheet_name='Avaliacoes_Startups')\n\nprint(f'Total de startups: {len(df)}')\nprint(f'Colunas: {df.shape[1]}')\nprint(f'\\nDistribuição por status:')\nprint(df['status'].value_counts())

In [ ]:
# Separar por status\ndf_falhas = df[df['status'] == 'Inativa'].copy()\ndf_investimento = df[df['status'] == 'Ativa'].copy()\n\nprint(f'Startups Ativas: {len(df_investimento)}')\nprint(f'Startups Inativas: {len(df_falhas)}')

## 2. Engenharia de Atributos

In [ ]:
# Identificar colunas de notas (1.1 a 8.5)\ncolunas_notas = [col for col in df.columns if col[0].isdigit() and '.' in col]\nprint(f'Colunas de avaliação: {len(colunas_notas)}')\nprint(f'Grupos: 1 a 8 (5 indicadores cada)')\n\n# Criar Score_Global\ndf_investimento['Score_Global'] = df_investimento[colunas_notas].mean(axis=1)\n\nprint(f'\\nScore Global médio (Ativas): {df_investimento["Score_Global"].mean():.2f}')

## 3. Visão Macro - Agregação por Dimensões

In [ ]:
# Definir dimensões\ndimensoes = {\n    'Grupo 1 - Performance Técnica': [col for col in colunas_notas if col.startswith('1.')],\n    'Grupo 2 - Viabilidade Econômica': [col for col in colunas_notas if col.startswith('2.')],\n    'Grupo 3 - Confiabilidade': [col for col in colunas_notas if col.startswith('3.')],\n    'Grupo 4 - Usabilidade': [col for col in colunas_notas if col.startswith('4.')],\n    'Grupo 5 - Eficiência Energética': [col for col in colunas_notas if col.startswith('5.')],\n    'Grupo 6 - Robustez Física': [col for col in colunas_notas if col.startswith('6.')],\n    'Grupo 7 - Conectividade': [col for col in colunas_notas if col.startswith('7.')],\n    'Grupo 8 - Sustentabilidade': [col for col in colunas_notas if col.startswith('8.')]\n}\n\n# Calcular médias por dimensão\nfor nome_dim, colunas_dim in dimensoes.items():\n    df_investimento[nome_dim] = df_investimento[colunas_dim].mean(axis=1)\n\n# Estatísticas por dimensão\nstats_dimensoes = df_investimento[[col for col in df_investimento.columns if 'Grupo' in col]].describe()\nstats_dimensoes

## 4. Decisão de Investimento

In [ ]:
# Calcular Score Performance + Viabilidade\ndf_investimento['Score_Performance_Viabilidade'] = (\n    df_investimento['Grupo 1 - Performance Técnica'] + \n    df_investimento['Grupo 2 - Viabilidade Econômica']\n) / 2\n\n# Identificar melhor startup\nmelhor_idx = df_investimento['Score_Performance_Viabilidade'].idxmax()\nmelhor = df_investimento.loc[melhor_idx]\n\nprint('🏆 MELHOR STARTUP PARA INVESTIMENTO')\nprint('=' * 60)\nprint(f'Nome: {melhor["nome_startup"]}')\nprint(f'Setor: {melhor["setor"]}')\nprint(f'Score Global: {melhor["Score_Global"]:.2f}/5.00')\nprint(f'Performance Técnica: {melhor["Grupo 1 - Performance Técnica"]:.2f}/5.00')\nprint(f'Viabilidade Econômica: {melhor["Grupo 2 - Viabilidade Econômica"]:.2f}/5.00')\nprint(f'Score Combinado: {melhor["Score_Performance_Viabilidade"]:.2f}/5.00')

In [ ]:
# Top 5 startups\ntop5 = df_investimento.nlargest(5, 'Score_Performance_Viabilidade')[\n    ['nome_startup', 'setor', 'Score_Global', 'Score_Performance_Viabilidade']\n]\ntop5

## 5. Visualizações

### 5.1 Gráfico de Radar - Melhor Startup (Plotly)

In [ ]:
# Preparar dados\ncategorias = list(dimensoes.keys())\nvalores = [melhor[cat] for cat in categorias]\n\n# Criar radar\nfig = go.Figure()\n\nfig.add_trace(go.Scatterpolar(\n    r=valores,\n    theta=categorias,\n    fill='toself',\n    name=melhor['nome_startup'],\n    line=dict(color='#1f77b4', width=2),\n    fillcolor='rgba(31, 119, 180, 0.3)'\n))\n\nfig.update_layout(\n    polar=dict(radialaxis=dict(visible=True, range=[0, 5])),\n    title=f\"Análise Multidimensional - {melhor['nome_startup']}\",\n    showlegend=True,\n    height=600\n)\n\nfig.show()

### 5.2 Boxplot - Score Global por Setor

In [ ]:
plt.figure(figsize=(12, 6))\nsns.boxplot(data=df_investimento, x='setor', y='Score_Global', palette='Set2')\nplt.title('Distribuição do Score Global por Setor\\n(Startups Ativas)', \n          fontsize=16, fontweight='bold')\nplt.xlabel('Setor', fontsize=12)\nplt.ylabel('Score Global', fontsize=12)\nplt.xticks(rotation=45, ha='right')\nplt.grid(axis='y', alpha=0.3)\nplt.tight_layout()\nplt.show()

### 5.3 Scatter Plot - Performance vs Viabilidade

In [ ]:
plt.figure(figsize=(12, 8))\n\n# Plot por setor\nfor setor in df_investimento['setor'].unique():\n    dados = df_investimento[df_investimento['setor'] == setor]\n    plt.scatter(\n        dados['Grupo 1 - Performance Técnica'],\n        dados['Grupo 2 - Viabilidade Econômica'],\n        label=setor,\n        s=100,\n        alpha=0.6\n    )\n\n# Destacar melhor\nplt.scatter(\n    melhor['Grupo 1 - Performance Técnica'],\n    melhor['Grupo 2 - Viabilidade Econômica'],\n    s=500,\n    c='gold',\n    marker='*',\n    edgecolors='red',\n    linewidth=2,\n    label=f'🏆 {melhor[\"nome_startup\"]}',\n    zorder=10\n)\n\nplt.title('Performance Técnica vs Viabilidade Econômica', fontsize=16, fontweight='bold')\nplt.xlabel('Grupo 1 - Performance Técnica', fontsize=12)\nplt.ylabel('Grupo 2 - Viabilidade Econômica', fontsize=12)\nplt.legend(loc='best')\nplt.grid(True, alpha=0.3)\nplt.tight_layout()\nplt.show()

## 6. Conclusões\n\n### Recomendação Final\n\nA análise multidimensional de 45 startups IoT identificou **ChiLeak** (setor Water-Ops) como a melhor opção de investimento.\n\n**Principais Destaques:**\n- Score Global: 3.98/5.00 (acima da média)\n- Melhor equilíbrio entre Performance Técnica e Viabilidade Econômica\n- Forte desempenho em todas as 8 dimensões avaliadas\n- Setor Water-Ops com alto potencial de crescimento\n\n**Próximos Passos:**\n1. Due diligence detalhada da ChiLeak\n2. Análise de mercado do setor Water-Ops\n3. Avaliação de riscos regulatórios\n4. Negociação de termos de investimento